In [1]:
import tensorflow.compat.v1 as tf

tf.disable_v2_behavior()
import os
import pickle
import numpy as np

CIFAR_DIR = "./cifar-10-batches-py"
print(os.listdir(CIFAR_DIR))

Instructions for updating:
non-resource variables are not supported in the long term
['batches.meta', 'data_batch_1', 'data_batch_2', 'data_batch_3', 'data_batch_4', 'data_batch_5', 'readme.html', 'test_batch']


In [4]:
def load_data(filename):
    """read data from data file."""
    with open(filename,'rb') as f:
        data = pickle.load(f,encoding='iso-8859-1')
        return data['data'],data['labels']

#tensorflow.Dataset
class CifarData:
    def __init__(self,filenames,need_shuffle):
        all_data = []
        all_labels = []
        for filename in filenames:
            data,labels = load_data(filename)
            all_data.append(data)
            all_labels.append(labels)
        self._data = np.vstack(all_data)
        self._data = self._data / 127.5 - 1
        self._labels = np.hstack(all_labels)
        print(self._data.shape)
        print(self._labels.shape)
        self._num_examples = self._data.shape[0]
        self._need_shuffle = need_shuffle
        self._indicator = 0
        if self._need_shuffle:
            self._shuffle_data()
    def _shuffle_data(self):
        p = np.random.permutation(self._num_examples)
        self._data = self._data[p]
        self._labels = self._labels[p]

    def next_batch(self,batch_size):
        """:return batch_szie examples as a batch."""
        end_indicator = self._indicator + batch_size
        if end_indicator > self._num_examples:
            if self._need_shuffle:
                self._shuffle_data()
                self._indicator = 0
                end_indicator = batch_size
            else:
                raise Exception("have no more examples")
        if end_indicator > self._num_examples:
            raise Exception("batch size is larger than all examples")
        batch_data = self._data[self._indicator:end_indicator]
        batch_labels = self._labels[self._indicator:end_indicator]
        self._indicator = end_indicator
        return batch_data,batch_labels

train_filenames = [os.path.join(CIFAR_DIR, 'data_batch_%d' % i ) for i in range(1,6)]
test_filenames = [os.path.join(CIFAR_DIR, 'test_batch')]

train_data = CifarData(train_filenames,True)
test_data = CifarData(test_filenames,True)

(50000, 3072)
(50000,)
(10000, 3072)
(10000,)


In [5]:
X = tf.placeholder(tf.float32,[None,3072])
# [None], eg: [0,5,6,3]
y = tf.placeholder(tf.int64,[None])

#(3072,10)
w = tf.get_variable('w',[X.get_shape()[-1],10],
                    initializer=tf.random_normal_initializer(0,1))

#(10,)
b = tf.get_variable('b',[10],
                    initializer=tf.constant_initializer(0.0))

#[None,3072]*[3072,10] = [None,10]
y_ = tf.matmul(X,w) + b

#mean square loss
"""
# e^x / sum(e^x)
# [[0.0.1,0.9,...,0.03],[]]
p_y = tf.nn.softmax(y_)
# 5 -> [0,0,0,0,1,0,0,0,0,0]
y_one_hot = tf.one_hot(y,10,dtype = tf.float32)
loss = tf.reduce_mean(tf.square(y_one_hot - p_y))
"""
loss = tf.losses.spares_softmax_cross_entropy(labels=y, logits=y_)
# y_ -> softmax
# y -> one_hot
# loss = ylogy_

# indices
predict = tf.argmax(y_,1)
# [1,0,1,1,1,0,0,0]
correct_prediction = tf.equal(predict,y)
accuary = tf.reduce_mean(tf.cast(correct_prediction,tf.float64))

with tf.name_scope('train_op'):
    train_op = tf.train.AdamOptimizer(1e-3).minimize(loss)

In [7]:
init = tf.global_variables_initializer()
batch_size = 20
train_steps = 100000
test_steps = 100
with tf.Session() as sess:
    sess.run(init)
    for i in range(train_steps):
        batch_data,batch_labels = train_data.next_batch(batch_size)
        loss_val,accu_val ,_ =sess.run(
            [loss,accuary,train_op],
            feed_dict={
                X:batch_data,
                y:batch_labels})
        if (i+1) % 500 ==0:
            print('[Train ] Step :%d, loss: %4.5f, acc: %4.5f'\
                  %(i+1,loss_val,accu_val))

        if (i+1) % 5000 == 0:
            test_data = CifarData(test_filenames, False)
            all_test_acc_val = []
            for j in range(test_steps):
                test_batch_data ,test_batch_labels\
                    = test_data.next_batch(batch_size)
                test_acc_val = sess.run(
                    [accuary],
                    feed_dict = {
                        X: test_batch_data,
                        y: test_batch_labels
                    })
                all_test_acc_val.append(test_acc_val)
            test_acc = np.nanmean(all_test_acc_val)
            print('[Test ] Step :%d, acc: %4.5f'\
                  %(i+1,test_acc))

[Train ] Step :500, loss: 0.17804, acc: 0.10000
[Train ] Step :1000, loss: 0.15592, acc: 0.20000
[Train ] Step :1500, loss: 0.14765, acc: 0.25000
[Train ] Step :2000, loss: 0.14376, acc: 0.25000
[Train ] Step :2500, loss: 0.14944, acc: 0.25000
[Train ] Step :3000, loss: 0.10833, acc: 0.45000
[Train ] Step :3500, loss: 0.12046, acc: 0.40000
[Train ] Step :4000, loss: 0.12668, acc: 0.35000
[Train ] Step :4500, loss: 0.15815, acc: 0.20000
[Train ] Step :5000, loss: 0.13896, acc: 0.25000
(10000, 3072)
(10000,)
[Test ] Step :5000, acc: 0.29150
[Train ] Step :5500, loss: 0.14112, acc: 0.25000
[Train ] Step :6000, loss: 0.14639, acc: 0.25000
[Train ] Step :6500, loss: 0.13635, acc: 0.30000
[Train ] Step :7000, loss: 0.11997, acc: 0.40000
[Train ] Step :7500, loss: 0.13256, acc: 0.30000
[Train ] Step :8000, loss: 0.10843, acc: 0.45000
[Train ] Step :8500, loss: 0.13638, acc: 0.30000
[Train ] Step :9000, loss: 0.11987, acc: 0.40000
[Train ] Step :9500, loss: 0.10484, acc: 0.45000
[Train ] Step 